In [2]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
df = pd.read_csv("/cluster/home/rodrigm/IonosphereModel/outputs/model_2025-05-01-14-35-02.csv")

In [4]:
df.head(1)

,sm_lat_ipp,sm_lon_ipp_sin,sm_lon_ipp_cos,sod_sin,sod_cos,satazi_cos,satazi_sin,satele,doy,year,prediction,target
0,-27.818573,0.863745,-0.503929,0.0,1.0,-0.768429,-0.639935,49.678001,183.0,2024.0,10.247264,8.231


In [5]:
pred = np.array(df["prediction"])
target = np.array(df["target"])
df["error"] = np.array(pred-target)

In [6]:
bias = np.mean(pred-target)
bias

np.float64(-0.2568654732013867)

In [7]:
mae = np.mean(np.abs(pred-target))
mae

np.float64(3.785403967862881)

In [8]:
np.mean(np.abs(pred-target-bias))

np.float64(3.7845541636484468)

In [9]:
error = pred-target

In [10]:
np.max(error)

np.float64(204.75806427001953)

In [11]:
center = []
num = []
for box_left in np.arange(-75,75.05,0.1):
    box_right = box_left + 0.1

    center.append((box_left+box_right)/2)
    num.append(np.sum(np.logical_and(box_left < error,error <= box_right)))

In [12]:
np.sum(num)/len(df)

np.float64(0.9994607646722719)

In [13]:
plot_df = pd.DataFrame()
plot_df["center"] = center
plot_df["num"] = num
alt.Chart(plot_df).mark_bar().encode(
    x="center",
    y="num"
)

alt.Chart(...)

In [14]:
angle = (np.array(df["sod_sin"]) >= 0)*np.arccos(np.array(df["sod_cos"])) + (np.array(df["sod_sin"]) < 0)*(2*np.pi-np.arccos(np.array(df["sod_cos"])))

In [15]:
sod =  (86400*angle) / (2 * np.pi)
hour = sod / 3600

In [16]:
absolute_error = np.abs(error)

In [28]:
h_list = []
mae_list = []
average_target = []
target_std = []
for h in np.arange(0,24,0.25):
    minute_window = np.logical_and(hour >= h, hour < h+0.5)

    h_list.append(h)
    mae_list.append(np.mean(absolute_error[minute_window]))
    average_target.append(np.mean(target[minute_window]))
    target_std.append(np.std(target[minute_window]))

In [18]:
plot_df = pd.DataFrame()
plot_df["quarter-hour"] = h_list
plot_df["mae"] = mae_list
alt.Chart(plot_df).mark_circle().encode(
    x="quarter-hour",
    y="mae"
)

alt.Chart(...)

In [19]:
plot_df = pd.DataFrame()
plot_df["quarter-hour"] = h_list
plot_df["mae"] = mae_list
plot_df["average_target"] = average_target

alt.Chart(plot_df).mark_circle().encode(
    x="quarter-hour",
    y="average_target"
)

alt.Chart(...)

In [30]:
plot_df = pd.DataFrame()
plot_df["quarter-hour"] = h_list
plot_df["mae"] = mae_list
plot_df["target-std"] = target_std

alt.Chart(plot_df).mark_circle().encode(
    x="quarter-hour",
    y="target-std"
)

alt.Chart(...)

In [20]:
angle = (np.array(df["sm_lon_ipp_sin"]) >= 0)*np.arccos(np.array(df["sm_lon_ipp_cos"])) + (np.array(df["sm_lon_ipp_sin"]) < 0)*(2*np.pi-np.arccos(np.array(df["sm_lon_ipp_cos"])))
angle *= 360 / (2*np.pi)

In [21]:
np.min(angle), np.max(angle)

(np.float64(0.0), np.float64(360.0))

In [22]:
np.min(df["satele"]), np.max(df["satele"])

(np.float64(5.0), np.float64(89.98799896240234))

In [23]:
ele = np.array(df["satele"])

In [24]:
ele_list = []
mae_list = []
sample_size = []
for a in np.arange(5,90,1):
    ele_window = np.logical_and(ele >= a, ele < a+1)
    sample_size.append(np.sum(ele_window))
    ele_list.append(a)
    mae_list.append(np.mean(absolute_error[ele_window]))

In [25]:
for a in np.arange(5,90,1):
    ele_window = np.logical_and(ele >= a, ele < a+1)
    print(a,a+1,np.sum(ele_window))

5 6 54601
6 7 60252
7 8 63913
8 9 66482
9 10 69779
10 11 71051
11 12 72456
12 13 72267
13 14 73717
14 15 73668
15 16 74107
16 17 73749
17 18 73168
18 19 73352
19 20 72655
20 21 71166
21 22 70098
22 23 68855
23 24 67818
24 25 67464
25 26 65513
26 27 63781
27 28 62163
28 29 61679
29 30 61212
30 31 61369
31 32 59912
32 33 59384
33 34 59313
34 35 58305
35 36 57068
36 37 55260
37 38 55218
38 39 53070
39 40 53120
40 41 51863
41 42 50359
42 43 49158
43 44 48745
44 45 47399
45 46 45582
46 47 45260
47 48 44149
48 49 43530
49 50 42631
50 51 41741
51 52 41191
52 53 39945
53 54 38007
54 55 37225
55 56 36162
56 57 34482
57 58 33983
58 59 33390
59 60 32069
60 61 31110
61 62 30994
62 63 29619
63 64 27969
64 65 27568
65 66 27333
66 67 25831
67 68 24378
68 69 23531
69 70 22285
70 71 20877
71 72 20480
72 73 19070
73 74 18133
74 75 17433
75 76 16055
76 77 15831
77 78 14807
78 79 13859
79 80 12197
80 81 10999
81 82 9560
82 83 8458
83 84 7241
84 85 6067
85 86 5115
86 87 3882
87 88 2252
88 89 1470
89 90 554

In [26]:
plot_df = pd.DataFrame()
plot_df["elevation angle"] = ele_list
plot_df["mae"] = mae_list
plot_df["sample_size"] = sample_size
alt.Chart(plot_df).mark_circle().encode(
    x="elevation angle",
    y="mae"
)

alt.Chart(...)

In [27]:

alt.Chart(plot_df).mark_circle().encode(
    x="elevation angle",
    y="sample_size"
)

alt.Chart(...)